In [1]:
import json
import numpy 
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import pyarrow
import sys
from tqdm import tqdm

from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path
from typing import List,Dict, Tuple
from collections import defaultdict
pd.set_option("display.max_columns", None)

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../scripts")))

import create_queries
import prepare_data

from bespoke.inventory.analysis.shared import download_util, inventory_types
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util

%load_ext autoreload
%autoreload 2

/Users/victoria/GitHub/venvs/bespoke/lib/python3.8/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
from underwriting import data_quality_checks

In [3]:
TRANSFER_PACKAGES_START_DATE = '2020-01-01'
SALES_TRANSACTIONS_START_DATE = '2020-01-01'
CURRENT_MONTH = '2022-01'

In [10]:
company_identifier_list = ['RA','HPCC','SFVPC','EMA','EMF','EMM','EMT','GRG','ST','EL']

In [8]:
def fetch_data(company_identifier,transfer_packages_start_date,sales_transactrions_start_date,current_month):
    df_in,df_sales_deduped,bad_dl,unknown_transfer,rwp,miss_incoming,miss_receipts = data_quality_checks.run(company_identifier,transfer_packages_start_date,sales_transactrions_start_date)    
    return df_in,df_sales_deduped
    

In [11]:
###test out
incoming_transfer_data,sales_transaction_deduped = fetch_data(['EL'],TRANSFER_PACKAGES_START_DATE,SALES_TRANSACTIONS_START_DATE,CURRENT_MONTH)


Verifying download summaries for license C9-0000370-LIC...
Earliest download summary: 2020-01-01
Latest download summary: 2022-02-15

[SUCCESS] All download summaries look good!
[SUCCESS] No unknown transfer packages!
97.0% of incoming transfer packages have receiver wholesale price
# transactions missing incoming transfer package: 22 (0.11358356136093757%)
# transactions total: 19369
# receipts with mismatching transactions: 0 (0.0%)
# receipts missing transactions: 0 (0.0%)
# receipts total: 7569


In [13]:
sales_transaction_deduped.head()

,license_number,receipt_number,rt_type,sales_customer_type,sales_datetime,sales_month,total_packages,rt_total_price,tx_type,tx_package_id,tx_package_label,tx_product_name,tx_product_category_name,tx_unit_of_measure,tx_quantity_sold,tx_total_price
0,C9-0000370-LIC,0165026072,active,Consumer,2022-02-14 20:52:42+00:00,2022-02,2,60.0,active,21023428,1A406030002B8E1000006961,FPB150-B8e,Flower (packaged eighth - each),Each,1.0,30.0
1,C9-0000370-LIC,0165026072,active,Consumer,2022-02-14 20:52:42+00:00,2022-02,2,60.0,active,21023431,1A406030002B8E1000006963,GMO59-B8e,Flower (packaged eighth - each),Each,1.0,30.0
2,C9-0000370-LIC,0165026071,active,Consumer,2022-02-14 19:47:48+00:00,2022-02,4,205.0,active,20358475,1A4060300022B79000204403,"Buddies 10:10 THC/CBD Capsule 50pc , Hashtag D...",Capsule (weight - each),Each,1.0,60.0
3,C9-0000370-LIC,0165026071,active,Consumer,2022-02-14 19:47:48+00:00,2022-02,4,205.0,active,18181096,1A40603000092E5000286522,Almora Farm: 1.2g Jar: Sugar: Sour Berry [H],Other Concentrate (weight - each),Each,2.0,70.0
4,C9-0000370-LIC,0165026071,active,Consumer,2022-02-14 19:47:48+00:00,2022-02,4,205.0,active,21841382,1A406030000A429000324130,STIIIZY - Blue Dream Curated Live Resin Sauce ...,Extract (weight - each),Each,1.0,40.0
